In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/ML/Kaggle_Janestreet/"
%cd /content/drive/MyDrive/ML/Kaggle_Janestreet/

%run ./Functions.ipynb

Mounted at /content/drive
/content/drive/MyDrive/ML/Kaggle_Janestreet


### Read Data

In [2]:
train = pd.read_parquet('./Data/jane_street_train.parquet')
display(train.shape)
display(train.head(2))

(2390491, 138)

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 138 entries, date to ts_id
dtypes: float64(135), int64(3)
memory usage: 2.5 GB


In [4]:
train.date.nunique()

500

In [ ]:
features = [c for c in train.columns if 'feature' in c]

train[features] = train[features].fillna(method = 'ffill').fillna(0)
train['action'] = ((train['resp_1'] > 0) & (train['resp_2'] > 0) & (train['resp_3'] > 0) & (train['resp_4'] > 0) & (train['resp'] > 0)).astype('int')
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
X = train[features].values
y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

date = train['date'].values
weight = train['weight'].values
resp = train['resp'].values

sw = np.mean(np.abs(train[resp_cols].values), axis = 1)

In [6]:
n_splits = 5
group_gap = 31

# Training

In [7]:
params = {'num_columns': len(features), 
          'num_labels': 5, 
          'hidden_units': [96, 96, 896, 448, 448, 256], 
          'dropout_rates': [0.03527936123679956, 0.038424974585075086, 0.42409238408801436, 0.10431484318345882, 0.49230389137187497, 0.32024444956111164, 0.2716856145683449, 0.4379233941604448], 
          'ls': 0, 
          'lr':1e-3, 
         }

In [8]:
scores = []
batch_size = 4096
gkf = PurgedGroupTimeSeriesSplit(n_splits = n_splits, group_gap = group_gap)
for fold, (tr, te) in enumerate(gkf.split(train['action'].values, train['action'].values, train['date'].values)):
    ckp_path = f'JSModel_{fold}.hdf5'
    model = cust_mlp(**params)
    ckp = ModelCheckpoint(ckp_path, monitor = 'val_action_AUC', verbose = 0, 
                          save_best_only = True, save_weights_only = True, mode = 'max')
    es = EarlyStopping(monitor = 'val_action_AUC', min_delta = 1e-4, patience = 10, mode = 'max', 
                        baseline = None, restore_best_weights = True, verbose = 0)
    history = model.fit(X[tr], [X[tr], y[tr], y[tr]], validation_data = (X[te], [X[te], y[te], y[te]]), 
                        sample_weight = sw[tr], 
                        epochs = 100, batch_size = batch_size, callbacks = [ckp, es], verbose = 0)
    hist = pd.DataFrame(history.history)
    score = hist['val_action_AUC'].max()
    print(f'Fold {fold} ROC AUC: {score:4f}\t')
    scores.append(score)

    K.clear_session()
    del model
    rubbish = gc.collect()

print(f'Weighted Average CV Score: {weighted_average(scores):.4f}')

Fold 0 ROC AUC: 0.527494	
Fold 1 ROC AUC: 0.532619	
Fold 2 ROC AUC: 0.534572	
Fold 3 ROC AUC: 0.534559	
Fold 4 ROC AUC: 0.545595	
Weighted Average CV Score: 0.5395
